## Hardware Details
[GCP](https://cloud.google.com/) VM: [n1-highmem-16](https://cloud.google.com/compute/docs/machine-types#n1_machine_types) (16 vCPUs, 104 GB memory)

In [1]:
%%bash
lscpu

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                16
On-line CPU(s) list:   0-15
Thread(s) per core:    2
Core(s) per socket:    8
Socket(s):             1
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 63
Model name:            Intel(R) Xeon(R) CPU @ 2.30GHz
Stepping:              0
CPU MHz:               2300.000
BogoMIPS:              4600.00
Hypervisor vendor:     KVM
Virtualization type:   full
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              46080K
NUMA node0 CPU(s):     0-15
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc eagerfpu pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hyperviso

In [2]:
%%bash
cat /proc/meminfo | head -n1

MemTotal:       107091244 kB


## Basic functions

In [3]:
import pandas as pd
import numpy as np
import random
import string
import gc
from itertools import product

In [4]:
def createTable(rowCount):
    gc.collect()
    t= pd.DataFrame({'weight': np.random.uniform(0, 2, rowCount),
                     'qty': np.random.randint(100, size=rowCount, dtype='int16'),
                     'risk': np.random.randint(10, size=rowCount, dtype='int16')})
    t['bucket'] = pd.Series(pd.Categorical.from_codes(np.random.choice(len(string.ascii_lowercase) ** 2, rowCount),
                                                      categories= map(''.join, product(string.ascii_lowercase, repeat=2))))
    return t

In [5]:
def executeQueryJoin(t):
    res = t.groupby('bucket').agg({'bucket': len, 'qty': [sum, np.mean], 'risk': [sum, np.mean]})
    res.columns = res.columns.map('_'.join)
    return res.rename(columns={'bucket_len':'NR', 'qty_sum':'TOTAL_QTY','qty_mean':'AVG_QTY', 
                        'risk_sum':'TOTAL_RISK','risk_mean':'AVG_RISK'}).join(
        t.groupby('bucket').apply(lambda g: np.average(g.qty, weights=g.weight)).to_frame('W_AVG_QTY')).join(
        t.groupby('bucket').apply(lambda g: np.average(g.risk, weights=g.weight)).to_frame('W_AVG_RISK'))


In [6]:
def my_agg(x):
    data = {'NR': x.bucket.count(),
            'TOTAL_QTY': x.qty.sum(),
            'AVG_QTY': x.qty.mean(),
            'TOTAL_RISK': x.risk.sum(),
            'AVG_RISK': x.risk.mean(),
            'W_AVG_QTY':  np.average(x.qty, weights=x.weight),
            'W_AVG_RISK':  np.average(x.risk, weights=x.weight)
           }
    return pd.Series(data, index=['NR', 'TOTAL_QTY', 'AVG_QTY', 'TOTAL_RISK', 
                                  'AVG_RISK', 'W_AVG_QTY', 'W_AVG_RISK'])

def executeQueryApply(t):
    return t.groupby('bucket').apply(my_agg).astype(
        {'NR': 'int64', 'TOTAL_QTY': 'int64', 'TOTAL_RISK': 'int64'})

## Row Number 10k

In [7]:
t = createTable(10 * 1000)

In [8]:
%timeit executeQueryJoin(t)

801 ms ± 8.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit executeQueryApply(t)

1.24 s ± 6.07 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Row Number 100k

In [10]:
del t
t = createTable(100 * 1000)

In [11]:
%timeit executeQueryJoin(t)

841 ms ± 9.71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%timeit executeQueryApply(t)

1.25 s ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Row Number 1M

In [13]:
del t
t = createTable(1000 * 1000)

In [14]:
%timeit executeQueryJoin(t)

1.3 s ± 13.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit executeQueryApply(t)

1.4 s ± 13.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Row Number 10M

In [16]:
del t
t = createTable(10 * 1000 * 1000)

In [17]:
%timeit executeQueryJoin(t)

8.21 s ± 83.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit executeQueryApply(t)

3.83 s ± 24.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Row Number 100M

In [19]:
del t
t = createTable(100 * 1000 * 1000)

In [20]:
%timeit -n 1 -r 10 executeQueryJoin(t)

1min 25s ± 1.01 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [21]:
%timeit -n 1 -r 10  executeQueryApply(t)

31.8 s ± 527 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


## Row Number 1B

In [22]:
del t
t = createTable(1000 * 1000 * 1000)

In [23]:
%timeit -n 1 -r 10 executeQueryJoin(t)

16min 12s ± 13.9 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [24]:
%timeit -n 1 -r 10 executeQueryApply(t)

5min 48s ± 2.82 s per loop (mean ± std. dev. of 10 runs, 1 loop each)
